In [49]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

# Procesamiento de datos

Leer datos

In [50]:
customers = pd.read_csv("data/customers.csv")
sales = pd.read_csv("data/sales.csv")
weekly_weather = pd.read_pickle("data/weather_grouped.pkl")
weather_clusters = pd.read_pickle("data/clusters_agglomerative.pkl")

Especificar productos de interés

In [51]:
prods = [
    "COLAS REGULAR FAMILIAR RETORNABLE",
    "COLAS REGULAR FAMILIAR NO RETORNABLE",
    "SABORES FAMILIAR RETORNABLE",
    "SABORES FAMILIAR NO RETORNABLE"
]

sales = sales[sales["prod_key"].isin(prods)]

Tomar los subcanales de los clientes

In [52]:
subchannels = customers[["customer_id", "comercial_subchannel_d"]]

Filtrar ventas de los top subcanales

In [53]:
# Merge de ventas con sus respectivos subcanales de acuerdo al cliente
sales_subchannels = sales.merge(subchannels, on="customer_id", how="left")

# Obtener los 4 subcanales con mayores ventas
sales_by_subchannel = sales_subchannels.groupby("comercial_subchannel_d")["sold"].sum().reset_index().sort_values(by="sold", ascending=False)
top4_subchannels = sales_by_subchannel.sort_values(by="sold", ascending=False).head(4)["comercial_subchannel_d"].tolist()

# Filtrar las ventas para que solo contengan los 4 subcanales con mayores ventas
customers_top4 = customers[customers["comercial_subchannel_d"].isin(top4_subchannels)]
sales_top4 = sales.merge(customers_top4[["customer_id", "comercial_subchannel_d"]], on="customer_id", how="inner")
sales_top4

,customer_id,prod_key,week,prediction,sold,comercial_subchannel_d
0,c93884c4c397a219a0fcb0061e15ced3359f7270294266...,COLAS REGULAR FAMILIAR NO RETORNABLE,202427,0.00,0.00,Abarrotes / Almacenes / Bodegas / Víveres
1,c93884c4c397a219a0fcb0061e15ced3359f7270294266...,SABORES FAMILIAR NO RETORNABLE,202440,0.88,3.52,Abarrotes / Almacenes / Bodegas / Víveres
2,c93884c4c397a219a0fcb0061e15ced3359f7270294266...,SABORES FAMILIAR RETORNABLE,202452,0.00,0.00,Abarrotes / Almacenes / Bodegas / Víveres
3,3115945bebb64d75821d8bc73777f3cdf3eede6c1352d6...,SABORES FAMILIAR RETORNABLE,202450,0.00,0.00,Estanquillos / kioscos
4,e04adcef4fba36fb00de00aab19bed307559265ebd917a...,COLAS REGULAR FAMILIAR RETORNABLE,202402,36.63,38.04,Estanquillos / kioscos
...,...,...,...,...,...,...
354635,59dc5e415352557a3742e469ed22c6ec95c5542f1d8a1b...,SABORES FAMILIAR NO RETORNABLE,202427,0.00,0.00,Abarrotes / Almacenes / Bodegas / Víveres
354636,648454febac3d78a467ca420b9683e909a1a2f3d2f5a92...,COLAS REGULAR FAMILIAR NO RETORNABLE,202404,5.28,20.43,Hogar con Venta
354637,648454febac3d78a467ca420b9683e909a1a2f3d2f5a92...,COLAS REGULAR FAMILIAR RETORNABLE,202448,27.47,20.78,Hogar con Venta
354638,92d6452328950102e9be4756f46982fc88e02fd9549d7e...,SABORES FAMILIAR NO RETORNABLE,202427,0.00,0.00,Hogar con Venta


Hacer pivot de las ventas

In [54]:
# Crear un diccionario que mapee cada prod_key a sus iniciales
prod_iniciales = {
    p: ''.join([palabra[0].upper() for palabra in p.split()])
    for p in sales_top4["prod_key"].unique()
}

# Pivot
sales_pivot = sales_top4.pivot_table(
    index=["customer_id", "week"],
    columns="prod_key",
    values=["prediction", "sold"]
)

# Renombrar columnas con iniciales
sales_pivot.columns = [
    f"{metric}_{prod_iniciales[product]}"
    for metric, product in sales_pivot.columns
]

sales_pivot = sales_pivot.reset_index()
sales_pivot

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR
0,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202401,6.34,7.75,1.32,0.0,5.28,7.40,0.00,0.00
1,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202402,6.34,5.64,0.00,0.0,10.70,14.79,2.95,0.00
2,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202403,3.17,6.87,0.00,0.0,5.28,7.40,6.34,0.00
3,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202404,5.28,4.23,0.00,0.0,5.99,10.92,0.00,0.00
4,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202405,3.17,7.75,0.00,0.0,5.99,9.86,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
88655,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202448,16.56,8.81,0.00,0.0,15.15,3.17,0.00,0.00
88656,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202449,16.56,9.86,0.00,0.0,25.01,9.86,0.00,2.11
88657,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202450,17.61,9.86,0.00,0.0,21.49,3.17,13.03,0.00
88658,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202451,21.13,9.86,0.00,0.0,16.91,13.39,0.00,0.00


Convertir datos de temperatura a Celsius

In [55]:
temp_cols = ["highest_temp", "lowest_temp", "avg_daily_all_hours", "avg_daily_max_min", "extreme_highest"]

weekly_weather[temp_cols] = (weekly_weather[temp_cols]  - 32) * 5/9

Agregar la estación a los datos de clima

In [56]:
weekly_weather = weather_clusters.merge(weekly_weather, on="weather_station_id", how="left")
weekly_weather["week"] = weekly_weather["week"].astype(int)
weekly_weather

,weather_station_id,cluster,week,highest_temp,lowest_temp,avg_daily_all_hours,avg_daily_max_min,extreme_highest,hdd_max_min,hdd_all_hours,cdd_max_min,cdd_all_hours,precipitation
0,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,0,202401,23.412698,6.428571,14.841270,14.920635,30.634921,6.142857,6.285714,0.000000,0.000000,0.011429
1,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,0,202402,25.000000,6.904762,15.317460,16.031746,31.190476,4.428571,5.428571,0.000000,0.000000,0.000000
2,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,0,202403,26.269841,9.047619,16.904762,17.619048,30.714286,1.571429,2.571429,0.428571,0.000000,0.002857
3,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,0,202404,26.666667,7.619048,17.142857,17.301587,32.460317,2.428571,2.142857,0.000000,0.000000,0.000000
4,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,0,202405,25.634921,9.603175,17.380952,17.539683,32.301587,2.142857,2.142857,0.857143,0.428571,0.008571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...,0,202448,29.603175,6.111111,17.301587,17.857143,28.968254,1.285714,1.857143,0.000000,0.000000,0.000000
1712,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...,0,202449,25.952381,7.857143,16.428571,16.904762,28.253968,2.857143,3.428571,0.142857,0.000000,0.000000
1713,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...,0,202450,26.904762,6.507937,16.269841,16.746032,28.571429,3.000000,3.714286,0.000000,0.000000,0.000000
1714,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...,0,202451,24.682540,5.952381,14.603175,15.317460,27.777778,5.571429,6.714286,0.000000,0.000000,0.001429


Agregar la estación a los datos de ventas

In [57]:
stations = customers[["customer_id", "weather_station_id"]]
sales_pivot = sales_pivot.merge(stations, on="customer_id", how="left")
sales_pivot

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,weather_station_id
0,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202401,6.34,7.75,1.32,0.0,5.28,7.40,0.00,0.00,ba5f6ceebdb0e625388424a471b3f46291d04bf1987995...
1,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202402,6.34,5.64,0.00,0.0,10.70,14.79,2.95,0.00,ba5f6ceebdb0e625388424a471b3f46291d04bf1987995...
2,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202403,3.17,6.87,0.00,0.0,5.28,7.40,6.34,0.00,ba5f6ceebdb0e625388424a471b3f46291d04bf1987995...
3,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202404,5.28,4.23,0.00,0.0,5.99,10.92,0.00,0.00,ba5f6ceebdb0e625388424a471b3f46291d04bf1987995...
4,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202405,3.17,7.75,0.00,0.0,5.99,9.86,0.00,0.00,ba5f6ceebdb0e625388424a471b3f46291d04bf1987995...
...,...,...,...,...,...,...,...,...,...,...,...
88655,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202448,16.56,8.81,0.00,0.0,15.15,3.17,0.00,0.00,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...
88656,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202449,16.56,9.86,0.00,0.0,25.01,9.86,0.00,2.11,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...
88657,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202450,17.61,9.86,0.00,0.0,21.49,3.17,13.03,0.00,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...
88658,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202451,21.13,9.86,0.00,0.0,16.91,13.39,0.00,0.00,fd15fbdb96badcc3a102ee0c538601adbe23dfe42511f8...


Merge de ventas con clima

In [58]:
sales_weather = sales_pivot.merge(
    weekly_weather,
    on=["weather_station_id", "week"],
    how="left"
)
sales_weather

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,...,highest_temp,lowest_temp,avg_daily_all_hours,avg_daily_max_min,extreme_highest,hdd_max_min,hdd_all_hours,cdd_max_min,cdd_all_hours,precipitation
0,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202401,6.34,7.75,1.32,0.0,5.28,7.40,0.00,0.00,...,17.142857,-1.825397,7.936508,7.619048,28.333333,19.428571,18.714286,0.000000,0.0,0.000000
1,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202402,6.34,5.64,0.00,0.0,10.70,14.79,2.95,0.00,...,18.095238,-1.428571,8.571429,8.333333,29.603175,18.285714,17.571429,0.000000,0.0,0.000000
2,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202403,3.17,6.87,0.00,0.0,5.28,7.40,6.34,0.00,...,21.666667,3.888889,13.015873,12.777778,29.126984,10.142857,9.571429,0.000000,0.0,0.000000
3,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202404,5.28,4.23,0.00,0.0,5.99,10.92,0.00,0.00,...,17.857143,2.380952,10.714286,10.238095,29.682540,15.000000,13.714286,0.000000,0.0,0.017143
4,004a92cf90a4405fa13b3b48deb9855c6dd957ee7ddb9b...,202405,3.17,7.75,0.00,0.0,5.99,9.86,0.00,0.00,...,21.349206,5.158730,13.412698,13.253968,30.952381,9.285714,8.857143,0.000000,0.0,0.004286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88655,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202448,16.56,8.81,0.00,0.0,15.15,3.17,0.00,0.00,...,29.603175,6.111111,17.301587,17.857143,28.968254,1.285714,1.857143,0.000000,0.0,0.000000
88656,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202449,16.56,9.86,0.00,0.0,25.01,9.86,0.00,2.11,...,25.952381,7.857143,16.428571,16.904762,28.253968,2.857143,3.428571,0.142857,0.0,0.000000
88657,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202450,17.61,9.86,0.00,0.0,21.49,3.17,13.03,0.00,...,26.904762,6.507937,16.269841,16.746032,28.571429,3.000000,3.714286,0.000000,0.0,0.000000
88658,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202451,21.13,9.86,0.00,0.0,16.91,13.39,0.00,0.00,...,24.682540,5.952381,14.603175,15.317460,27.777778,5.571429,6.714286,0.000000,0.0,0.001429


Filtrar solamente los clusters previamente seleccionados

In [59]:
clusters = [0, 4, 5]
sales_data = sales_weather[sales_weather["cluster"].isin(clusters)]

Observamos que los datos de ventas y predicciones de sabores no son muy buenos. La mayoría de los datos son 0.

In [60]:
sales_data.describe()

,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,cluster,highest_temp,lowest_temp,avg_daily_all_hours,avg_daily_max_min,extreme_highest,hdd_max_min,hdd_all_hours,cdd_max_min,cdd_all_hours,precipitation
count,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000,74776.000000
mean,202426.500000,16.053423,26.574999,1.667484,0.154481,17.865696,27.167346,3.945564,0.625757,1.940195,29.515307,14.660287,21.842439,22.109060,36.279573,1.250831,1.258219,8.118772,7.574610,0.072949
std,15.008431,25.112056,35.968812,3.870472,0.989794,30.957778,38.867046,6.767957,2.286468,2.158037,4.684451,5.996017,4.898072,4.903704,3.820946,2.652135,2.690216,7.389727,7.245874,0.156846
min,202401.000000,0.000000,0.000000,0.000000,0.000000,-7.040000,-16.200000,-3.870000,-6.690000,0.000000,10.238095,-2.063492,4.126984,4.206349,27.380952,0.000000,0.000000,0.000000,0.000000,0.000000
25%,202413.750000,2.110000,7.570000,0.000000,0.000000,3.170000,7.040000,0.000000,0.000000,0.000000,26.666667,10.238095,18.571429,18.650794,33.015873,0.000000,0.000000,1.714286,1.285714,0.000000
50%,202426.500000,8.810000,18.320000,0.000000,0.000000,9.510000,17.610000,1.590000,0.000000,0.000000,28.968254,14.365079,21.190476,21.587302,36.666667,0.000000,0.000000,6.142857,5.142857,0.000000
75%,202439.250000,20.250000,34.520000,1.760000,0.000000,21.130000,34.870000,5.640000,0.000000,4.000000,32.619048,18.015873,25.476190,25.634921,39.523810,1.142857,1.142857,13.285714,12.857143,0.060000
max,202452.000000,782.490000,1114.300000,55.480000,35.220000,1117.480000,1396.620000,209.710000,105.640000,5.000000,43.174603,28.888889,35.079365,34.841270,45.000000,25.857143,25.571429,30.000000,30.142857,1.460000


In [61]:
df = sales_data.copy()

target_cols = [
    "prediction_CRFNR", "prediction_CRFR", "prediction_SFNR", "prediction_SFR",
    "sold_CRFNR", "sold_CRFR", "sold_SFNR", "sold_SFR"
]

results = []
total_rows = len(df)

for col in target_cols:
    zero_count = (df[col] == 0).sum()
    non_zero_count = total_rows - zero_count
    
    zero_proportion = (zero_count / total_rows) * 100
    non_zero_proportion = (non_zero_count / total_rows) * 100
    
    results.append({
        "Columna": col,
        "Total de Registros": total_rows,
        "Ceros (Cuenta)": zero_count,
        "No Ceros (Cuenta)": non_zero_count,
        "Ceros (%)": round(zero_proportion, 2),
        "No Ceros (%)": round(non_zero_proportion, 2)
    })

df_zeros = pd.DataFrame(results)

df_zeros

,Columna,Total de Registros,Ceros (Cuenta),No Ceros (Cuenta),Ceros (%),No Ceros (%)
0,prediction_CRFNR,74776,15988,58788,21.38,78.62
1,prediction_CRFR,74776,8112,66664,10.85,89.15
2,prediction_SFNR,74776,51272,23504,68.57,31.43
3,prediction_SFR,74776,72273,2503,96.65,3.35
4,sold_CRFNR,74776,12567,62209,16.81,83.19
5,sold_CRFR,74776,8144,66632,10.89,89.11
6,sold_SFNR,74776,32082,42694,42.90,57.10
7,sold_SFR,74776,65721,9055,87.89,12.11


In [62]:
sales_data.head()

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,...,highest_temp,lowest_temp,avg_daily_all_hours,avg_daily_max_min,extreme_highest,hdd_max_min,hdd_all_hours,cdd_max_min,cdd_all_hours,precipitation
52,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202401,8.63,34.52,6.34,0.0,0.00,33.46,1.41,0.0,...,23.412698,6.428571,14.841270,14.920635,30.634921,6.142857,6.285714,0.000000,0.000000,0.011429
53,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202402,6.52,33.64,4.93,0.0,8.98,39.19,4.23,0.0,...,25.000000,6.904762,15.317460,16.031746,31.190476,4.428571,5.428571,0.000000,0.000000,0.000000
54,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202403,8.37,34.52,4.93,0.0,11.62,40.16,8.45,0.0,...,26.269841,9.047619,16.904762,17.619048,30.714286,1.571429,2.571429,0.428571,0.000000,0.002857
55,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202404,8.72,32.58,2.11,0.0,11.62,39.10,4.23,0.0,...,26.666667,7.619048,17.142857,17.301587,32.460317,2.428571,2.142857,0.000000,0.000000,0.000000
56,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202405,10.30,32.58,4.93,0.0,8.98,37.69,9.86,0.0,...,25.634921,9.603175,17.380952,17.539683,32.301587,2.142857,2.142857,0.857143,0.428571,0.008571


## Forecast de clima

In [63]:
weather_forecast = pd.read_pickle("data/tablaDatosForecast_id.pkl")
weather_forecast = weather_forecast[weather_forecast.columns[:-8]]
weather_forecast["week"] = weather_forecast["week"].astype(int)
weather_forecast.head()

,cluster,weather_station_id,week,temp_mean_week,temp_std_week,temp_max_week,temp_min_week,temp_range_mean,days_above_30,precip_total_week,...,hdd_week,temp_precip_interaction,temp_first_last_diff,temp_max_jump,temp_volatility_index,temp_increase_days,temp_drop_days,temp_trend,temp_reversal,heat_wave_start
0,0,004784ca59d476279540ee2fb4f01b1521c38a3cd643cc...,202401,14.166667,0.582672,24.722222,4.351852,20.370370,0,0.0,...,23.000000,14.166667,0.000000,1.111111,0.666667,2,2,-0.142857,0.0,0
1,0,694278a0d7919878b55f79d5ffc32517621cbc29c96a17...,202401,12.777778,0.860663,21.666667,5.648148,16.018519,0,0.0,...,31.333333,12.777778,1.111111,2.222222,1.111111,2,2,0.253968,-1.0,0
2,0,93e7e9504b1dba5af6f251b1a494f5e95c98a154a39175...,202401,11.018519,1.548263,18.240741,5.648148,12.592593,0,0.0,...,41.888889,11.018519,-2.222222,3.333333,1.777778,1,2,-0.269841,1.0,0
3,0,ab1a0c25e39afe3a7199859e7ed34780f17691e0e363ee...,202401,9.074074,1.092436,20.277778,-1.203704,21.481481,0,0.0,...,53.555556,9.074074,-1.666667,2.777778,1.444444,2,2,-0.285714,1.0,0
4,0,d0345419341b3009182cc1252378c4bd86f19012dd85f9...,202401,14.166667,0.464811,23.703704,6.574074,17.129630,0,0.0,...,23.000000,14.166667,-0.555556,1.111111,0.333333,1,1,-0.174603,1.0,0


In [64]:
sales_data_f = sales_pivot.merge(
    weather_forecast,
    on=["weather_station_id", "week"],
    how="inner"
)
sales_data_f

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,...,hdd_week,temp_precip_interaction,temp_first_last_diff,temp_max_jump,temp_volatility_index,temp_increase_days,temp_drop_days,temp_trend,temp_reversal,heat_wave_start
0,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202401,8.63,34.52,6.34,0.0,0.00,33.46,1.41,0.00,...,23.000000,14.166667,0.000000,1.111111,0.666667,2,2,-0.142857,0.0,0
1,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202402,6.52,33.64,4.93,0.0,8.98,39.19,4.23,0.00,...,15.222222,15.462963,0.555556,1.111111,0.777778,3,2,0.142857,-1.0,0
2,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202403,8.37,34.52,4.93,0.0,11.62,40.16,8.45,0.00,...,2.333333,14.891975,0.555556,1.111111,0.777778,3,2,0.142857,-1.0,0
3,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202404,8.72,32.58,2.11,0.0,11.62,39.10,4.23,0.00,...,6.333333,16.944444,-1.111111,0.555556,0.222222,0,2,-0.206349,1.0,0
4,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202405,10.30,32.58,4.93,0.0,8.98,37.69,9.86,0.00,...,9.666667,13.657407,-2.222222,1.666667,0.888889,2,3,-0.492063,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74771,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202448,16.56,8.81,0.00,0.0,15.15,3.17,0.00,0.00,...,12.444444,15.925926,-1.666667,1.666667,0.777778,1,2,-0.349206,1.0,0
74772,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202449,16.56,9.86,0.00,0.0,25.01,9.86,0.00,2.11,...,14.111111,13.040123,-0.555556,0.555556,0.111111,0,1,-0.079365,1.0,0
74773,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202450,17.61,9.86,0.00,0.0,21.49,3.17,13.03,0.00,...,11.888889,16.018519,-0.555556,1.111111,0.555556,1,2,-0.142857,1.0,0
74774,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202451,21.13,9.86,0.00,0.0,16.91,13.39,0.00,0.00,...,23.555556,14.074074,-2.222222,1.111111,0.444444,0,3,-0.412698,1.0,0


Etiquetar tama;o de cliente

In [ ]:
sold_cols = ['sold_CRFNR', 'sold_CRFR', 'sold_SFNR', 'sold_SFR']
df = sales_data_f

# Calcular las ventas totales por semana (para todos los productos)
df['total_sold_week'] = df[sold_cols].sum(axis=1)

# Calcular el promedio de ventas semanales por cliente (Tamaño)
customer_size_df = df.groupby('customer_id').agg(
    avg_total_sold=('total_sold_week', 'mean'),
    cluster_fixed=('cluster', 'first') # Mantener el clúster
).reset_index()
customer_size_df

,customer_id,avg_total_sold,cluster_fixed
0,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,60.396731,0
1,006a0380c0d745c36f9a282779e2a83b53b8bd14ecb18b...,44.091923,5
2,00b9021f1bff2ea5f5de6a3c676b77441b95971c7af2fe...,79.547308,5
3,00d441f7a1510ed4675b1da7191c5c77d9d15c38b6daf7...,137.016538,4
4,00f04b8b8b740caeb337f8a94705ee5b78e7096027e8d7...,19.543846,4
...,...,...,...
1433,fe72ea21f031d057c522096ba4f884b1a9013bdcebf669...,8.748077,0
1434,fecf7eb55e5e726970ac12c69172d4a6852dd1523ea5dc...,255.412115,0
1435,ff4629eb1b5d775ba47e53c467f068b8052f3bc0e659c5...,23.071154,4
1436,ff8d48231b74fde7ba4fdc890b0d9ce94475b344a9914c...,70.147308,0


In [66]:
customer_size_df.describe()

,avg_total_sold,cluster_fixed
count,1438.000000,1438.000000
mean,49.604363,1.940195
std,56.702155,2.158774
min,0.000000,0.000000
25%,20.031394,0.000000
50%,34.734712,0.000000
75%,59.011731,4.000000
max,1221.945577,5.000000


In [67]:
bins = [-np.inf, 20, 59, 100, np.inf] 
labels = ['Pequeno', 'Mediano', 'Grande', 'Muy Grande']
customer_size_df['client_size'] = pd.cut(
    customer_size_df['avg_total_sold'], 
    bins=bins, 
    labels=labels, 
    right=True,
    include_lowest=True
)

In [68]:
customer_size = customer_size_df[['customer_id', 'client_size']].copy()

sales_data_f = pd.merge(
    sales_data_f, 
    customer_size, 
    on='customer_id', 
    how='left'
)
sales_data_f

,customer_id,week,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,sold_SFR,...,temp_first_last_diff,temp_max_jump,temp_volatility_index,temp_increase_days,temp_drop_days,temp_trend,temp_reversal,heat_wave_start,total_sold_week,client_size
0,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202401,8.63,34.52,6.34,0.0,0.00,33.46,1.41,0.00,...,0.000000,1.111111,0.666667,2,2,-0.142857,0.0,0,34.87,Grande
1,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202402,6.52,33.64,4.93,0.0,8.98,39.19,4.23,0.00,...,0.555556,1.111111,0.777778,3,2,0.142857,-1.0,0,52.40,Grande
2,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202403,8.37,34.52,4.93,0.0,11.62,40.16,8.45,0.00,...,0.555556,1.111111,0.777778,3,2,0.142857,-1.0,0,60.23,Grande
3,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202404,8.72,32.58,2.11,0.0,11.62,39.10,4.23,0.00,...,-1.111111,0.555556,0.222222,0,2,-0.206349,1.0,0,54.95,Grande
4,0066c9224e55b41e3d79e1b494bb34b863859c3d476ff2...,202405,10.30,32.58,4.93,0.0,8.98,37.69,9.86,0.00,...,-2.222222,1.666667,0.888889,2,3,-0.492063,1.0,0,56.53,Grande
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74771,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202448,16.56,8.81,0.00,0.0,15.15,3.17,0.00,0.00,...,-1.666667,1.666667,0.777778,1,2,-0.349206,1.0,0,18.32,Mediano
74772,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202449,16.56,9.86,0.00,0.0,25.01,9.86,0.00,2.11,...,-0.555556,0.555556,0.111111,0,1,-0.079365,1.0,0,36.98,Mediano
74773,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202450,17.61,9.86,0.00,0.0,21.49,3.17,13.03,0.00,...,-0.555556,1.111111,0.555556,1,2,-0.142857,1.0,0,37.69,Mediano
74774,ffed8a62de208d58e9b1c5b6f29fee28608b3f0b2e44d5...,202451,21.13,9.86,0.00,0.0,16.91,13.39,0.00,0.00,...,-2.222222,1.111111,0.444444,0,3,-0.412698,1.0,0,30.30,Mediano


In [69]:
# Columnas que deben ser sumadas
product_suffixes = ['CRFNR', 'CRFR', 'SFNR', 'SFR']
pred_cols = [f'prediction_{p}' for p in product_suffixes]
sold_cols = [f'sold_{p}' for p in product_suffixes]
cols_to_sum = pred_cols + sold_cols

# Columnas a excluir
exclusion_list = ['customer_id', 'weather_station_id', 'total_sold_week', 'cluster', 'client_size', 'week'] + cols_to_sum

# Determinar las Columnas de Clima para promediar
climate_cols = [col for col in sales_data_f.columns if col not in exclusion_list]
                
agg_dict = {}

# A. Sumar ventas y predicciones
for col in cols_to_sum:
    agg_dict[col] = 'sum'

# B. Promediar columnas de clima
for col in climate_cols:
    agg_dict[col] = 'mean'

In [70]:
weekly_sales_segmented = sales_data_f.groupby(
    ['week', 'cluster', 'client_size']
).agg(
    agg_dict
).reset_index()
weekly_sales_segmented

C:\Users\sofia\AppData\Local\Temp\ipykernel_24980\1332537366.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weekly_sales_segmented = sales_data_f.groupby(


,week,cluster,client_size,prediction_CRFNR,prediction_CRFR,prediction_SFNR,prediction_SFR,sold_CRFNR,sold_CRFR,sold_SFNR,...,hdd_week,temp_precip_interaction,temp_first_last_diff,temp_max_jump,temp_volatility_index,temp_increase_days,temp_drop_days,temp_trend,temp_reversal,heat_wave_start
0,202401,0,Pequeno,715.65,1227.05,94.43,0.00,701.37,1171.96,237.90,...,31.252525,12.791246,-0.166667,1.371212,0.755556,1.704545,1.986364,-0.080952,0.204545,0.0
1,202401,0,Mediano,3853.89,7140.45,763.03,6.34,3274.84,6280.20,967.11,...,29.928432,13.011928,-0.232526,1.329713,0.741019,1.744361,1.979950,-0.113816,0.248120,0.0
2,202401,0,Grande,2017.51,3796.23,449.61,0.00,1745.25,3006.84,539.02,...,29.974697,13.004217,-0.308031,1.375138,0.774477,1.752475,2.000000,-0.128713,0.267327,0.0
3,202401,0,Muy Grande,2911.74,5604.44,656.10,0.00,2295.73,4887.59,652.45,...,31.431373,12.761438,-0.383987,1.397059,0.779412,1.676471,2.000000,-0.127918,0.352941,0.0
4,202401,4,Pequeno,391.06,468.28,35.12,3.17,296.87,383.77,57.42,...,21.166028,9.833972,-2.126437,3.422733,1.893997,1.632184,2.816092,-0.354315,0.885057,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,202452,4,Muy Grande,3747.11,4178.02,412.63,55.27,5374.01,4598.67,1026.43,...,1.427702,17.657069,-0.646880,1.590563,0.768645,2.328767,1.794521,-0.091542,-0.260274,0.0
620,202452,5,Pequeno,143.00,244.93,0.00,0.00,296.57,245.37,44.00,...,1.487179,18.757123,-2.606838,1.559829,0.717949,0.730769,2.461538,-0.595849,0.730769,0.0
621,202452,5,Mediano,1039.79,1496.49,34.17,7.04,1683.50,1653.17,210.46,...,1.105072,18.779187,-2.192029,1.364734,0.636473,0.804348,2.434783,-0.509834,0.717391,0.0
622,202452,5,Grande,743.22,1081.54,58.13,3.52,976.05,901.90,167.66,...,1.533333,18.759259,-2.685185,1.537037,0.700000,0.666667,2.400000,-0.607407,0.700000,0.0


In [71]:
sales_data_f.to_pickle("data/sales_data_f2.pkl")

In [72]:
weekly_sales_segmented.to_pickle("data/weekly_sales_segmented.pkl")